In [ ]:
import tensorflow.keras as keras
from tensorflow.keras import layers
import tensorflow as tf
import tensorflow_datasets as tfds
import time
import os
import re
import numpy as np
from pathlib import Path

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
drive_path = "gdrive/MyDrive/MachineLearning/HandsOnMachineLearning/chapter15"

Mounted at /content/gdrive


# 9.

In [ ]:
DOWNLOAD_ROOT = "http://download.tensorflow.org/data/"
FILENAME = "quickdraw_tutorial_dataset_v1.tar.gz"
filepath = keras.utils.get_file(FILENAME, DOWNLOAD_ROOT + FILENAME, extract=True)
path = Path(filepath).parent

1065301781/1065301781 [==============================] - 49s 0us/step


In [ ]:
def get_filepaths(split, path):
    regex = re.compile(split + ".tfrecord-\d{5}-of-\d{5}")
    filepaths = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if re.match(regex, file):
                filepaths.append(str(path / file))
    return filepaths

In [ ]:
train_filepaths = get_filepaths("training", path)
eval_filepaths = get_filepaths("eval", path)
print(train_filepaths)

['/root/.keras/datasets/training.tfrecord-00004-of-00010', '/root/.keras/datasets/training.tfrecord-00001-of-00010', '/root/.keras/datasets/training.tfrecord-00008-of-00010', '/root/.keras/datasets/training.tfrecord-00009-of-00010', '/root/.keras/datasets/training.tfrecord-00003-of-00010', '/root/.keras/datasets/training.tfrecord-00000-of-00010', '/root/.keras/datasets/training.tfrecord-00007-of-00010', '/root/.keras/datasets/training.tfrecord-00006-of-00010', '/root/.keras/datasets/training.tfrecord-00005-of-00010', '/root/.keras/datasets/training.tfrecord-00002-of-00010']


In [ ]:
def list_record_features(tfrecord_path):
    for rec in tf.data.TFRecordDataset(tfrecord_path).skip(100).take(10):
        example_bytes = rec.numpy()
        example = tf.train.Example()
        example.ParseFromString(example_bytes)
        for key, value in example.features.feature.items():
            kind = value.WhichOneof("kind")
            size = len(getattr(value, kind).value)
            print(f"key:  {key}", f"kind: {kind}", f"size: {size}",
                  sep="\n", end="\n\n")


list_record_features(train_filepaths[3])

DataLossError: {{function_node __wrapped__IteratorGetNext_output_types_1_device_/job:localhost/replica:0/task:0/device:CPU:0}} corrupted record at 0 (Is this even a TFRecord file?) [Op:IteratorGetNext] name: 

In [ ]:
feature_description = {
    "class_index": tf.io.FixedLenFeature([], tf.int64),
    "shape": tf.io.FixedLenFeature([2], tf.int64),
    "ink": tf.io.VarLenFeature(tf.float32)
}
MAX_LENGTH = 100


def _parse_function(proto):
    example = tf.io.parse_single_example(proto, feature_description)
    class_index = example["class_index"]
    ink = example["ink"]
    ink = tf.sparse.to_dense(ink)
    ink = tf.reshape(ink, (-1, 3))
    return ink[:MAX_LENGTH], class_index


def image_generator(tfrecord_filepaths, max_length=100):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_filepaths)
    dataset = raw_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)

    for batch in dataset.padded_batch(batch_size, padded_shapes=([100, 3], [])).prefetch(1):
        yield batch


batch_size = 32
output_signature = (
    tf.TensorSpec(shape=(None, None, 3), dtype=tf.float32),
    tf.TensorSpec(shape=(None,), dtype=tf.int64)
)
train_set = tf.data.Dataset.from_generator(
    lambda: image_generator(train_filepaths),
    output_signature=output_signature
)
valid_set = tf.data.Dataset.from_generator(
    lambda: image_generator(eval_filepaths[:5]),
    output_signature=output_signature
)
test_set = tf.data.Dataset.from_generator(
    lambda: image_generator(eval_filepaths[5:]),
    output_signature=output_signature
)

In [ ]:
with open(path / "eval.tfrecord.classes") as test_classes_file:
    test_classes = test_classes_file.readlines()

with open(path / "training.tfrecord.classes") as train_classes_file:
    train_classes = train_classes_file.readlines()

assert train_classes == test_classes
class_names = [name.strip().lower() for name in train_classes]

In [ ]:
filepath = time.strftime(f"{drive_path}/models/sketchrnn_%Y_%m_%d-%H_%M_%S")
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
    initial_value_threshold=None
)


model = keras.models.Sequential([
    keras.layers.Conv1D(32, kernel_size=5, strides=2, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(64, kernel_size=5, strides=2, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(128, kernel_size=3, strides=2, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.LSTM(128, return_sequences=True),
    keras.layers.LSTM(128),
    keras.layers.Dense(len(class_names), activation="softmax")
])
optimizer = keras.optimizers.SGD(learning_rate=1e-2, clipnorm=1.)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])

In [ ]:
# 3,450,000 training size
model.fit(train_set, epochs=2, validation_data=valid_set,
          callbacks=[checkpoint_cb])

# 10.

In [ ]:
DOWNLOAD_ROOT = "https://github.com/ageron/handson-ml2/raw/master/datasets/jsb_chorales/"
FILENAME = "jsb_chorales.tgz"
filepath = keras.utils.get_file(FILENAME, DOWNLOAD_ROOT + FILENAME, extract=True)
path = Path(filepath).parent

In [ ]:
train_filepaths = sorted([str(record_path) for record_path in path.glob("train/chorale_*.csv")])
valid_filepaths = sorted([str(record_path) for record_path in path.glob("valid/chorale_*.csv")])
test_filepaths = sorted([str(record_path) for record_path in path.glob("test/chorale_*.csv")])

In [ ]:
import pandas as pd


def create_chorales(filepaths):
    return [pd.read_csv(filepath).values.tolist() for filepath in filepaths]


train_chorales = create_chorales(train_filepaths)
valid_chorales = create_chorales(valid_filepaths)
test_chorales = create_chorales(test_filepaths)

In [ ]:
notes = set()
for chorales in (train_chorales, valid_chorales, test_chorales):
    for chorale in chorales:
        for chord in chorale:
            # |= is union operator
            notes |= set(chord)

n_notes = len(notes)
min_note = min(notes - {0})
max_note = max(notes)

assert min_note == 36
assert max_note == 81

In [ ]:
'''
Creates dataset from chorales, then uses it to create a flattened sequences
of notes, where the last note is used as label. The notes are also normalized
to range 0-46
'''
def create_target(batch):
    X = batch[:, :-1]
    Y = batch[:, 1:] # predict next note in each arpegio, at each step
    return X, Y

def preprocess(window):
    window = tf.where(window == 0, window, window - min_note + 1) # shift values
    return tf.reshape(window, [-1]) # convert to arpegio

def bach_dataset(chorales, batch_size=32, shuffle_buffer_size=None,
                 window_size=32, window_shift=16, cache=True):
    def batch_window(window):
        return window.batch(window_size + 1)

    def to_windows(chorale):
        dataset = tf.data.Dataset.from_tensor_slices(chorale)
        dataset = dataset.window(window_size + 1, window_shift, drop_remainder=True)
        return dataset.flat_map(batch_window)

    chorales = tf.ragged.constant(chorales, ragged_rank=1)
    dataset = tf.data.Dataset.from_tensor_slices(chorales)
    dataset = dataset.flat_map(to_windows).map(preprocess)
    if cache:
        dataset = dataset.cache()
    if shuffle_buffer_size:
        dataset = dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(create_target)
    return dataset.prefetch(1)


In [ ]:
train_set = bach_dataset(train_chorales, shuffle_buffer_size=1000)
valid_set = bach_dataset(valid_chorales)
test_set = bach_dataset(test_chorales)

In [ ]:
n_embedding_dims = 5

model = keras.models.Sequential([
    keras.layers.Embedding(input_dim=n_notes, output_dim=n_embedding_dims,
                           input_shape=[None]),
    keras.layers.Conv1D(32, kernel_size=2, padding="causal", activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(48, kernel_size=2, padding="causal", activation="relu", dilation_rate=2),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(64, kernel_size=2, padding="causal", activation="relu", dilation_rate=4),
    keras.layers.BatchNormalization(),
    keras.layers.Conv1D(96, kernel_size=2, padding="causal", activation="relu", dilation_rate=8),
    keras.layers.BatchNormalization(),
    keras.layers.LSTM(256, return_sequences=True),
    keras.layers.Dense(n_notes, activation="softmax")
])

model.summary()

In [ ]:
filepath = time.strftime(f"{drive_path}/models/bach_%Y_%m_%d-%H_%M_%S")
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',
    initial_value_threshold=None
)

optimizer = keras.optimizers.Nadam(learning_rate=1e-3)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
              metrics=["accuracy"])
model.fit(train_set, epochs=20, validation_data=valid_set,
          callbacks=[checkpoint_cb])

In [ ]:
model.evaluate(test_set)

In [ ]:
'''
Predict returns probabilites of notes for each position in sequence.
Argmax then gives the index (note) with the highest probability for each
position in sequence. The last element in this new sequence is the new note.
'''
def generate_chorale(model, seed_chords, length):
    arpegio = preprocess(tf.constant(seed_chords, dtype=tf.int64))
    arpegio = tf.reshape(arpegio, [1, -1])
    for chord in range(length):
        for note in range(4):
            #next_note = model.predict_classes(arpegio)[:1, -1:]
            next_note = np.argmax(model.predict(arpegio), axis=-1)[:1, -1:]
            arpegio = tf.concat([arpegio, next_note], axis=1)
    arpegio = tf.where(arpegio == 0, arpegio, arpegio + min_note - 1)
    return tf.reshape(arpegio, shape=[-1, 4])

'''
Predicts the probability of each note, scales each probability logarithmically,
then randomly selects index (note) from this probability distribution.
'''
def generate_chorale_v2(model, seed_chords, length, temperature=1):
    arpegio = preprocess(tf.constant(seed_chords, dtype=tf.int64))
    arpegio = tf.reshape(arpegio, [1, -1])
    for chord in range(length):
        for note in range(4):
            next_note_probas = model.predict(arpegio)[0, -1:]
            rescaled_logits = tf.math.log(next_note_probas) / temperature
            next_note = tf.random.categorical(rescaled_logits, num_samples=1)
            arpegio = tf.concat([arpegio, next_note], axis=1)
    arpegio = tf.where(arpegio == 0, arpegio, arpegio + min_note - 1)
    return tf.reshape(arpegio, shape=[-1, 4])

In [ ]:
from IPython.display import Audio

def notes_to_frequencies(notes):
    # Frequency doubles when you go up one octave; there are 12 semi-tones
    # per octave; Note A on octave 4 is 440 Hz, and it is note number 69.
    return 2 ** ((np.array(notes) - 69) / 12) * 440

def frequencies_to_samples(frequencies, tempo, sample_rate):
    note_duration = 60 / tempo # the tempo is measured in beats per minutes
    # To reduce click sound at every beat, we round the frequencies to try to
    # get the samples close to zero at the end of each note.
    frequencies = np.round(note_duration * frequencies) / note_duration
    n_samples = int(note_duration * sample_rate)
    time = np.linspace(0, note_duration, n_samples)
    sine_waves = np.sin(2 * np.pi * frequencies.reshape(-1, 1) * time)
    # Removing all notes with frequencies ≤ 9 Hz (includes note 0 = silence)
    sine_waves *= (frequencies > 9.).reshape(-1, 1)
    return sine_waves.reshape(-1)

def chords_to_samples(chords, tempo, sample_rate):
    freqs = notes_to_frequencies(chords)
    freqs = np.r_[freqs, freqs[-1:]] # make last note a bit longer
    merged = np.mean([frequencies_to_samples(melody, tempo, sample_rate)
                     for melody in freqs.T], axis=0)
    n_fade_out_samples = sample_rate * 60 // tempo # fade out last note
    fade_out = np.linspace(1., 0., n_fade_out_samples)**2
    merged[-n_fade_out_samples:] *= fade_out
    return merged

def play_chords(chords, tempo=160, amplitude=0.1, sample_rate=44100, filepath=None):
    samples = amplitude * chords_to_samples(chords, tempo, sample_rate)
    if filepath:
        from scipy.io import wavfile
        samples = (2**15 * samples).astype(np.int16)
        wavfile.write(filepath, sample_rate, samples)
        return display(Audio(filepath))
    else:
        return display(Audio(samples, rate=sample_rate))

In [ ]:
seed_chords = test_chorales[2][:8]
play_chords(seed_chords, amplitude=0.2)

In [ ]:
new_chorale = generate_chorale(model, seed_chords, 56)
play_chords(new_chorale)

In [ ]:
new_chorale_v2_medium = generate_chorale_v2(model, seed_chords, 56, temperature=1.0)
play_chords(new_chorale_v2_medium, filepath="bach_medium.wav")